In [72]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import re

In [73]:
# Number of rows
n_rows = 1500

# Generate random dates within a range
start_date = datetime(2023, 1, 1)
end_date = datetime(2025, 1, 1)
time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days

dates = [start_date + timedelta(days=random.randrange(days_between_dates)) for _ in range(n_rows)]

In [74]:
# Sample data for other columns
GID = ['FIC', 'SCI', 'THR', 'ROM', 'COM', 'BIO', 'HIS', 'POE']
vendors = ['Alpha Press', 'Beta Books', 'Gamma Publishing', 'Delta Inc.', 'Epsilon Ltd.','alpha press','Beta books','AlBe Publishing','Thomson Press']
stages = ['Manuscript', 'Proofreading', 'Typesetting', 'Printing', 'Binding']
feedback_areas = ['Front Cover', 'Back Cover', 'Body Text', 'Reference Section', 'Illustrations', 'Index']
sources = ['Internal', 'External']
feedback_types = ['Positive', 'Negative']
criticalities = ['High', 'Medium', 'Low']
actions = ['Revised', 'Accepted', 'Rejected', 'Pending', 'Addressed']

In [75]:
# Create the DataFrame
df = pd.DataFrame({
    'GID': [random.choice(GID) for _ in range(n_rows)],
    'DATE': dates,
    'VENDOR': [random.choice(vendors) for _ in range(n_rows)],
    'STAGE': [random.choice(stages) for _ in range(n_rows)],
    'FEEDBACK_AREA': [random.choice(feedback_areas) for _ in range(n_rows)], # Renamed and updated
    'SOURCE': [random.choice(sources) for _ in range(n_rows)],
    'FEEDBACK_TYPE': [random.choice(feedback_types) for _ in range(n_rows)],
    'CRITICALITY': [random.choice(criticalities) for _ in range(n_rows)],
    'ACTION_TAKEN_BY_VENDOR': [random.choice(actions) for _ in range(n_rows)]
})

In [76]:
# Generate GID (always a genre code)
genres = ['FIC', 'SCI', 'THR', 'ROM', 'COM', 'BIO', 'HIS', 'POE']
df['GID'] = [random.choice(genres) for _ in range(len(df))]  # Assign a random genre to each row

# Create 'instance' (rank within date)
df['instance'] = df.groupby('DATE').cumcount() + 1  # Start from 1

# Generate ID as GID&DATE&instance
df['ID'] = df.apply(lambda row: f"{row['GID']}{row['DATE'].strftime('%y%m%d')}{row['instance']}", axis=1)


In [77]:
# Generate GID (always a genre code)
genres = ['FIC', 'SCI', 'THR', 'ROM', 'COM', 'BIO', 'HIS', 'POE']
df['GID'] = [random.choice(genres) for _ in range(len(df))]  # Assign a random genre to each row

# Create 'instance' (rank within date)
df['instance'] = df.groupby('DATE').cumcount() + 1  # Start from 1

# Generate ID as GID&DATE&instance
df['ID'] = df.apply(lambda row: f"{row['GID']}{row['DATE'].strftime('%y%m%d')}{row['instance']}", axis=1)


In [78]:
# Introduce some data quality issues:

# 1. Missing values (Only in ACTION_TAKEN_BY_VENDOR)
df.loc[df.sample(frac=0.03).index, 'ACTION_TAKEN_BY_VENDOR'] = np.nan  # 3% missing actions
df.loc[df.sample(frac=0.02).index, 'FEEDBACK_AREA'] = 'front cover' #typo and space
df.loc[df.sample(frac=0.01).index, 'STAGE'] = 'proof reading' #typo and space
df.loc[df.sample(frac=0.01).index, 'FEEDBACK_AREA'] = 'Body' #Short form entry

In [79]:

# 3. Duplicate entries
df = pd.concat([df, df.sample(frac=0.01)], ignore_index=True) #1% duplicate rows

# 4. Outliers (e.g., unrealistic dates)
df.loc[df.sample(frac=0.01).index, 'DATE'] = datetime(2020, 1, 1) # old date

# 5. Imbalanced data (e.g., very few positive feedback)
positive_indices = df[df['FEEDBACK_TYPE'] == 'Positive'].index
negative_indices = df[df['FEEDBACK_TYPE'] == 'Negative'].index

# Adjust the ratio (e.g., 1% positive)
num_positive_to_keep = int(len(df) * 0.01)
positive_to_remove = random.sample(list(positive_indices), len(positive_indices) - num_positive_to_keep)
df.drop(positive_to_remove, inplace=True)

# The below scripts will be used for data cleaning purposes

In [80]:
#Start with reading the csv file that has the data
pd.read_csv(r'C:\Users\saifali\OneDrive - Wiley\Learning Resource\PythonProjects\feedback.csv',index_col=0)

,DATE,VENDOR,STAGE,FEEDBACK_AREA,SOURCE,FEEDBACK_TYPE,CRITICALITY,ACTION_TAKEN_BY_VENDOR,instance,ID
GID,,,,,,,,,,
SCI,2023-12-05,Alpha Press,Binding,Illustrations,Internal,Negative,Medium,Addressed,1,SCI2312051
POE,2024-11-04,Beta books,Typesetting,Body Text,External,Negative,High,Pending,1,POE2411041
POE,2023-01-10,Beta Books,Typesetting,Front Cover,Internal,Negative,Low,Revised,1,POE2301101
FIC,2024-04-03,Thomson Press,Typesetting,Front Cover,Internal,Negative,Medium,Rejected,1,FIC2404031
COM,2023-12-20,alpha press,Proofreading,Back Cover,Internal,Negative,Medium,Addressed,1,COM2312201
...,...,...,...,...,...,...,...,...,...,...
BIO,2024-07-16,AlBe Publishing,Proofreading,Index,Internal,Negative,Medium,Pending,1,BIO2407161
COM,2023-12-16,Thomson Press,Printing,Body Text,Internal,Positive,Low,Addressed,1,COM2312161
FIC,2024-10-17,Epsilon Ltd.,Typesetting,Body Text,Internal,Negative,Medium,Revised,1,FIC2410171


In [81]:
#Reordering the column name to ensure the table format remains same always and only rleveant fields are taken in
colummn_order=['ID','GID','DATE','VENDOR','STAGE','FEEDBACK_AREA','SOURCE','FEEDBACK_TYPE','CRITICALITY','ACTION_TAKEN_BY_VENDOR']
feedback=df[colummn_order]

In [82]:
feedback

,ID,GID,DATE,VENDOR,STAGE,FEEDBACK_AREA,SOURCE,FEEDBACK_TYPE,CRITICALITY,ACTION_TAKEN_BY_VENDOR
0,BIO2301081,BIO,2023-01-08,Thomson Press,Binding,Reference Section,External,Negative,Low,Rejected
1,FIC2402061,FIC,2024-02-06,Epsilon Ltd.,Typesetting,Reference Section,Internal,Negative,Low,Revised
2,BIO2311131,BIO,2023-11-13,alpha press,Manuscript,Reference Section,External,Negative,Medium,Revised
3,POE2408021,POE,2024-08-02,Beta Books,Printing,Back Cover,Internal,Negative,Low,Accepted
4,ROM2412091,ROM,2024-12-09,Thomson Press,Binding,Back Cover,External,Negative,Medium,Pending
...,...,...,...,...,...,...,...,...,...,...
1507,COM2408133,COM,2024-08-13,Beta books,Proofreading,Back Cover,External,Negative,Medium,Addressed
1509,ROM2403252,ROM,2024-03-25,Beta Books,Binding,Reference Section,Internal,Negative,Medium,Rejected
1510,SCI2311135,SCI,2023-11-13,Beta books,Proofreading,Body Text,Internal,Negative,Low,Pending
1511,SCI2406102,SCI,2024-06-10,Alpha Press,Printing,Body Text,Internal,Negative,Low,Addressed


In [83]:
#Checking if there are inconsistency with Vendor name by returning all uniques values this field holds
feedback.VENDOR.unique()

#This can be done by using some aggregation as well
feedback_by_vendor=feedback.groupby('VENDOR')['ID'].count()
print(feedback_by_vendor)

VENDOR
AlBe Publishing      94
Alpha Press         104
Beta Books           96
Beta books           95
Delta Inc.           64
Epsilon Ltd.         88
Gamma Publishing     64
Thomson Press        87
alpha press          77
Name: ID, dtype: int64


We will start by filtering our dataframe to show only relevant data:

*   **VENDOR:**
    *   Thomson Press needs to be excluded
*   **FEEDBACK_TYPE:**
    *   Only Negative feedback needs to be kept


In [84]:
#Ensuring only the data for relevant vendors are passed on for further steps by creating a list, removed 'Thomson Press' from the list
key_vendors=['Beta Books', 'Delta Inc.', 'Gamma Publishing', 'Alpha Press',
       'Epsilon Ltd.', 'AlBe Publishing', 'alpha press',
       'Beta books']

In [85]:
#creating a new dataframe that shows data for key vendors only
key_vendors_feedback = feedback[feedback['VENDOR'].isin(key_vendors)]

In [86]:
#creating a new dataframe that shows data for key vendors only
key_vendors_feedback = feedback[feedback['FEEDBACK_TYPE'].isin(['Negative'])]
key_vendors_feedback

,ID,GID,DATE,VENDOR,STAGE,FEEDBACK_AREA,SOURCE,FEEDBACK_TYPE,CRITICALITY,ACTION_TAKEN_BY_VENDOR
0,BIO2301081,BIO,2023-01-08,Thomson Press,Binding,Reference Section,External,Negative,Low,Rejected
1,FIC2402061,FIC,2024-02-06,Epsilon Ltd.,Typesetting,Reference Section,Internal,Negative,Low,Revised
2,BIO2311131,BIO,2023-11-13,alpha press,Manuscript,Reference Section,External,Negative,Medium,Revised
3,POE2408021,POE,2024-08-02,Beta Books,Printing,Back Cover,Internal,Negative,Low,Accepted
4,ROM2412091,ROM,2024-12-09,Thomson Press,Binding,Back Cover,External,Negative,Medium,Pending
...,...,...,...,...,...,...,...,...,...,...
1507,COM2408133,COM,2024-08-13,Beta books,Proofreading,Back Cover,External,Negative,Medium,Addressed
1509,ROM2403252,ROM,2024-03-25,Beta Books,Binding,Reference Section,Internal,Negative,Medium,Rejected
1510,SCI2311135,SCI,2023-11-13,Beta books,Proofreading,Body Text,Internal,Negative,Low,Pending
1511,SCI2406102,SCI,2024-06-10,Alpha Press,Printing,Body Text,Internal,Negative,Low,Addressed


In [87]:
# View the list of values in each field and check if there are any discrepancies
columns_to_check = ['GID', 'VENDOR', 'STAGE', 'FEEDBACK_AREA', 'SOURCE',
       'FEEDBACK_TYPE', 'CRITICALITY', 'ACTION_TAKEN_BY_VENDOR']  # Only the columns you want to see
for column in columns_to_check:
    print(f"\nUnique values in {column}:")
    print(key_vendors_feedback[column].unique())


Unique values in GID:
['BIO' 'FIC' 'POE' 'ROM' 'COM' 'THR' 'HIS' 'SCI']

Unique values in VENDOR:
['Thomson Press' 'Epsilon Ltd.' 'alpha press' 'Beta Books' 'Alpha Press'
 'Gamma Publishing' 'AlBe Publishing' 'Delta Inc.' 'Beta books']

Unique values in STAGE:
['Binding' 'Typesetting' 'Manuscript' 'Printing' 'Proofreading'
 'proof reading']

Unique values in FEEDBACK_AREA:
['Reference Section' 'Back Cover' 'Front Cover' 'Illustrations' 'Index'
 'Body Text' 'front cover' 'Body']

Unique values in SOURCE:
['External' 'Internal']

Unique values in FEEDBACK_TYPE:
['Negative']

Unique values in CRITICALITY:
['Low' 'Medium' 'High']

Unique values in ACTION_TAKEN_BY_VENDOR:
['Rejected' 'Revised' 'Accepted' 'Pending' 'Addressed' nan]


We can see that there are a few inconsistencies:

*   **VENDOR:**
    *   Variations in the name of Alpha and Beta.
    *   Further, they (Alpha and Beta) need to be grouped as AlBe Publishing (recent merger).
*   **STAGE:**
    *   Variations in how "Proofreading" is written.
*   **FEEDBACK_AREA:**
    *   Variations in how "Front Cover" is written.

In [88]:
# All variations of Alpha and Beta Press be grouped in AlBe Publishing
vendors_to_replace = ['Beta Books','Alpha Press','alpha press', 'Beta books']

for vendor in vendors_to_replace:
   key_vendors_feedback = key_vendors_feedback.replace(vendor, 'AlBe Publishing')

key_vendors_feedback.groupby('VENDOR')['ID'].count()

VENDOR
AlBe Publishing     456
Delta Inc.           63
Epsilon Ltd.         87
Gamma Publishing     63
Thomson Press        85
Name: ID, dtype: int64

In [89]:
# Replace proof reading with Proofreading
stage_to_replace = ['proof reading']

for stage in stage_to_replace:
   key_vendors_feedback = key_vendors_feedback.replace(stage, 'Proofreading')

key_vendors_feedback.groupby('STAGE')['ID'].count()

STAGE
Binding         149
Manuscript      164
Printing        160
Proofreading    148
Typesetting     133
Name: ID, dtype: int64

In [90]:
# Replace front cover with Front Cover
FEEDBACK_AREA_to_replace = ['front cover']

for FEEDBACK_AREA in FEEDBACK_AREA_to_replace:
   key_vendors_feedback = key_vendors_feedback.replace(FEEDBACK_AREA, 'Front Cover')

key_vendors_feedback.groupby('FEEDBACK_AREA')['ID'].count()

FEEDBACK_AREA
Back Cover           127
Body                   8
Body Text            119
Front Cover          136
Illustrations        121
Index                119
Reference Section    124
Name: ID, dtype: int64

In [ ]:
key_vendors_feedback.to_excel("feedback_data.xlsx", index=False)